d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# CSE Coding Assignment
## Instructions

- Please answer all questions
- You can use any language you wish (e.g. Python, Scala, SQL...)
- Several Markdown cells require completion. Please edit the Markdown cells to include your answer.
- Your final notebook should compile without errors when you click "Run All"

**Please do not publish questions. This is a confidential assignment.**

### Creating a Cluster

You will need to create a Databricks Cluster. More information on this process is available here: https://docs.databricks.com/user-guide/clusters/create.html

## Getting Started

**REQUIRED:** Run the following cells exactly as written to retrieve the necessary Coding Assignment Data Sets from Amazon S3.

In [0]:
%sh curl --remote-name-all 'https://files.training.databricks.com/assessments/cse-take-home/{covertype,kafka,treecover,u.data,u.item}.csv'


[1/5]: https://files.training.databricks.com/assessments/cse-take-home/covertype.csv --> covertype.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   133  100   133    0     0   1176      0 --:--:-- --:--:-- --:--:--  1176

[2/5]: https://files.training.databricks.com/assessments/cse-take-home/kafka.csv --> kafka.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  135k  100  135k    0     0  2119k      0 --:--:-- --:--:-- --:--:-- 2119k

[3/5]: https://files.training.databricks.com/assessments/cse-take-home/treecover.csv --> treecover.csv
  % Total    % Received % Xferd  Average Speed   Time    Time 

In [0]:
dbutils.fs.cp("file:/databricks/driver/covertype.csv", "dbfs:/FileStore/tmp/covertype.csv")
dbutils.fs.cp("file:/databricks/driver/kafka.csv", "dbfs:/FileStore/tmp/kafka.csv")
dbutils.fs.cp("file:/databricks/driver/treecover.csv", "dbfs:/FileStore/tmp/treecover.csv")
dbutils.fs.cp("file:/databricks/driver/u.data.csv", "dbfs:/FileStore/tmp/u.data.csv")
dbutils.fs.cp("file:/databricks/driver/u.item.csv", "dbfs:/FileStore/tmp/u.item.csv")

Out[2]: True

## Part 1: Reading and Parsing Data

### Question 1:  Code Challenge - Load a CSV

- Load the CSV file at `dbfs:/FileStore/tmp/nl/treecover.csv` into a DataFrame.
- Use Apache Spark to read in the data, assigned to the variable `treeCoverDF`.
- Your method to get the CSV file into Databricks isn't graded. We are only concerned with how you use Spark to parse and load the actual data. 
- Please use the `inferSchema` option.

In [0]:
path = "/FileStore/tmp/treecover.csv"

treeCoverDF=spark.read.option("header", "true").option("inferSchema" , "true").csv(path)
treeCoverDF.show()

+---+---------+------+-----+--------------------------------+------------------------------+-------------------------------+----------------------------------+----------+---------+---------------+---------+
| Id|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Horizontal_Distance_To_Fire_Points|Cover_Type|Soil_Type|Wilderness_Area|Hillshade|
+---+---------+------+-----+--------------------------------+------------------------------+-------------------------------+----------------------------------+----------+---------+---------------+---------+
|  1|     2596|    51|    3|                             258|                             0|                            510|                              6279|         5|       29|              1|      9am|
|  2|     2590|    56|    2|                             212|                            -6|                            390|                              6225|         5|  

### Question 2:  Code Challenge - Print the Schema

Use Apache Spark to display the Schema of the `treeCoverDF` Dataframe.

In [0]:
treeCoverDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Cover_Type: integer (nullable = true)
 |-- Soil_Type: integer (nullable = true)
 |-- Wilderness_Area: integer (nullable = true)
 |-- Hillshade: string (nullable = true)



### Question 3:  Code Challenge - Rows & Columns

Use Apache Spark to display the number of rows and columns in the DataFrame.

In [0]:
treeCoverDF.agg({'Elevation': 'count'}).show()
print("Number of rows in treeCoverDF: ", treeCoverDF.count())
print("Number of Columns in treeCoverDF: ", len(treeCoverDF.columns))

+----------------+
|count(Elevation)|
+----------------+
|           15120|
+----------------+

Number of rows in treeCoverDF:  15120
Number of Columns in treeCoverDF:  12


#Part 2: Analysis

### Question 4:  Code Challenge - Summary Statistics for a Feature

Use Apache Spark to answer these questions about the `treeCoverDF` DataFrame:
- What is the range - minimum and maximum - of values for the feature `elevation`?
- What are the mean and standard deviation of the feature `elevation`?

In [0]:
from pyspark.sql.functions import max,min # max/min method from pyspark.sql.functions resolved the issue of max/min value of column as it overrides built in python methods
treeCoverDF.agg(max(treeCoverDF.Elevation)).show()#1
treeCoverDF.agg(min(treeCoverDF.Elevation)).show()#2
#3treeCoverDF.agg({'Elevation': 'min'}).show()
#4treeCoverDF.agg({'Elevation': 'max'}).show()
#5treeCoverDF.select([min("Elevation")]).show()
#6treeCoverDF.select([max("Elevation")]).show()
#7treeCoverDF.registerTempTable("treeCoverDF_tbl")
#8spark.sql("SELECT MIN(Elevation) as minval_Of_Elevation FROM treeCoverDF_tbl").show()
#9spark.sql("SELECT MAX(Elevation) as maxval_Of_Elevation FROM treeCoverDF_tbl").show()

from pyspark.sql.functions import mean , stddev , col

df_stats = treeCoverDF.select(
    mean(col('Elevation')).alias('_mean'),
    stddev(col('Elevation')).alias('_std')
).collect()
print(df_stats[0]['_mean'],df_stats[0]['_std'])

+--------------+
|max(Elevation)|
+--------------+
|          3849|
+--------------+

+--------------+
|min(Elevation)|
+--------------+
|          1863|
+--------------+

2749.3225529100528 417.67818734804985


In [0]:
treeCoverDF.select([min("Elevation")]).show()
treeCoverDF.select([max("Elevation")]).show()

+--------------+
|min(Elevation)|
+--------------+
|          1863|
+--------------+

+--------------+
|max(Elevation)|
+--------------+
|          3849|
+--------------+



In [0]:
treeCoverDF.agg({'Elevation': 'min'}).show()
treeCoverDF.agg({'Elevation': 'max'}).show()

+--------------+
|min(Elevation)|
+--------------+
|          1863|
+--------------+

+--------------+
|max(Elevation)|
+--------------+
|          3849|
+--------------+



In [0]:
treeCoverDF.registerTempTable("treeCoverDF_tbl")
spark.sql("SELECT MIN(Elevation) as minval_Of_Elevation FROM treeCoverDF_tbl").show()
spark.sql("SELECT MAX(Elevation) as maxval_Of_Elevation FROM treeCoverDF_tbl").show()

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
+-------------------+
|minval_Of_Elevation|
+-------------------+
|               1863|
+-------------------+

+-------------------+
|maxval_Of_Elevation|
+-------------------+
|               3849|
+-------------------+



### Answer #4:

- Min `elevation`: `1863`
- Max `elevation`: `3849`
- Mean `elevation`: `2749.3225529100528`
- Standard Deviation of `elevation`: `417.67818734804985`

### Question 5:  Code Challenge - Record Count

Use Apache Spark to answer the following question:
- How many entries in the dataset have an `elevation` greater than or equal to 2749.32 meters **AND** a `Cover_Type` of 1 or 2?

In [0]:
#Be cautious abt '(' and ')' while using bitwise operators | / &
dfl=treeCoverDF.filter(((treeCoverDF["Cover_Type"] == 1) | (treeCoverDF["Cover_Type"] == 2)) & (treeCoverDF["Elevation"]>=2749.32)).count()
print(dfl)

3883


### Question 6: Code Challenge - Compute a Percentage

Use Apache Spark to answer the following question:
- What percentage of entries with `Cover_Type` 1 or 2 have an `elevation` at or above 2749.32 meters?

In [0]:
percent= dfl/treeCoverDF.count()*100
print(percent,"%")

25.681216931216934 %


### Question 7: Code Challenge - Visualize Feature Distribution

Use any [visualization tool available in the Databricks Runtime](https://docs.databricks.com/user-guide/visualizations/index.html) to generate the following visualization:

- a bar chart that helps visualize the distribution of different Wilderness Areas in our dataset

In [0]:
from pyspark.sql.functions import *

Wilderness_Aray = treeCoverDF.groupBy(treeCoverDF["Wilderness_Area"]).count()
display(Wilderness_Aray)

Wilderness_Area,count
1,3597
3,6349
4,4675
2,499


### Question 8: Code Challenge - Visualize Average Elevation by Cover Type 

Use any [visualization tool available in the Databricks Runtime](https://docs.databricks.com/user-guide/visualizations/index.html) to generate the following visualization:

- a bar chart showing the average elevation of each cover type with string labels for cover type

**NOTE: you will need to match the integer values in the column `treeCoverDF.Cover_Type` to the string values in `dbfs:/FileStore/tmp/nl/covertype.csv` to retrieve the Cover Type Labels. It is recommended to use an Apache Spark join.**

In [0]:
from pyspark.sql.functions import *
fft1=treeCoverDF.groupby(treeCoverDF["Cover_Type"]).mean().select("Cover_Type","avg(Elevation)")
path = "/FileStore/tmp/covertype.csv"
CovertypeDF=spark.read.option("header", "true").option("inferSchema" , "true").csv(path)
fft2=fft1.join(CovertypeDF).where(fft1["Cover_Type"]==CovertypeDF["cover_type_key"]).select("cover_type_label","avg(Elevation)")
display(fft2)

cover_type_label,avg(Elevation)
Spruce/Fir,3128.025925925926
Douglas-fir,2423.276851851852
Ponderosa Pine,2398.4231481481484
Aspen,2786.801388888889
Cottonwood/Willow,2223.42037037037
Krummholz,3362.7699074074076
Lodgepole Pine,2922.5402777777776


#Part 3: Data Ingestion, Cleansing, and Transformations

## Instructions 

This is a multi-step, data pipeline question in which you need to achieve a few objectives to build a successful job.

### Data Sets

#### `u.data.csv`

- The full u data set, 100000 ratings by 943 users on 1682 items. 
- Each user has rated at least 20 movies.  
- Users and items are numbered consecutively from 1. 
- The data is randomly ordered. 
- This is a tab separated file consisting of four columns: 
   - user id 
   - movie id 
   - rating 
   - date (unix seconds since 1/1/1970 UTC)

#### Desired schema

- `user_id INTEGER`
- `movie_id INTEGER`
- `rating INTEGER`
- `date DATE `

#### `u.item.csv`

- This is a `|` separated file consisting of six columns:
   - movie id
   - movie title
   - release date
   - video release date
   - IMDb URL
   - genre
- movie ids in this file match movie ids in `u.data`.

#### Desired schema

- `movie_id INTEGER`
- `movie_title STRING`

### Question 9:  Code Challenge - Load DataFrames

Use Apache Spark to perform the following:
1. define the correct schemas for each Data Set to be imported as described above  
   **note:** 
      - for `u.data.csv`, `date` *must* be stored using `DateType` with the format `yyyy-MM-dd`
      - you may need to ingest `timestamp` data using `IntegerType`
      - be sure to drop unneccesary columns for `u.item.csv`
1. import the two files as DataFrames names `uDataDF` and `uItemDF` using the schemas you defined and these paths:
   - `dbfs:/FileStore/tmp/u.data.csv`
   - `dbfs:/FileStore/tmp/u.item.csv`
1. order the `uDataDF` DataFrame by the `date` column

**NOTE:** Please display the DataFrames, `uDataDF` and `uItemDF` after loading.

#### `uDataDF`

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

path= "/FileStore/tmp/u.data.csv"
schema = StructType([StructField("User_id", IntegerType(), True),
                     StructField("Movie_id", IntegerType(), True),
                     StructField("Rating", IntegerType(), True),
                     StructField("date_time", IntegerType()),
                    ])
DF1=spark.read.option("delimiter", "\t").csv(path,schema=schema)
DF2=DF1.withColumn("date_time_fmt",from_unixtime(DF1.date_time))\
.select("User_id","Movie_id","Rating",date_format('date_time_fmt','yyyy-mm-dd').alias('Date'),date_format('date_time_fmt','hh:mm:ss').alias('Time'))
uDataDF=DF2.orderBy(["Date"],ascending=False)
#uDataDF=uDataDF.withColumn("Date",to_date("Date"))
uDataDF.display()

#DF1 = DF1.withColumn("date", DF1["date_time"].cast(TimestampType())) # Here typecasting is done on date_time to covert from unix to timestamp formats
#1uDataDF=pyspark.sql.functions.split(DF1['date_time'], ' ') ###Here split can be done but to obtain two different data frames is being created in #1,2,3
#2df_date = df.withColumn('Date', split_col.getItem(0))
#3df_time = df.withColumn('timestamp', split_col.getItem(1))


User_id,Movie_id,Rating,Date,Time
650,135,4,1998-59-31,09:59:05
7,609,3,1998-59-31,01:59:09
766,357,4,1998-59-31,01:59:18
454,984,3,1998-59-31,08:59:28
90,657,5,1998-59-31,10:59:50
7,537,3,1998-59-31,01:59:09
766,616,3,1998-59-31,01:59:49
766,82,3,1998-59-31,01:59:18
766,527,5,1998-59-31,01:59:18
650,519,4,1998-59-31,09:59:05


#### `uItemDF`

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

path= "/FileStore/tmp/u.item.csv"
schemat = StructType([StructField("Movie_id", IntegerType(), True),
                     StructField("Movie_title", StringType(), True)
                    ])
uItemDF=spark.read.option("delimiter", "|").csv(path,schema=schemat)
uItemDF.display()

Movie_id,Movie_title
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
7,Twelve Monkeys (1995)
8,Babe (1995)
9,Dead Man Walking (1995)
10,Richard III (1995)


### Question 10:  Code Challenge - Perform a Join

Use Apache Spark to do the following:
- join `uDataDF` and `uItemDf` on `movie_id` as a new DataFrame called `uMovieDF`  
   **note:** make sure you do not create duplicate `movie_id` columns
   
**NOTE:** Please display the DataFrame `uMovieDF`.

In [0]:
uMovieDF=uDataDF.join(uItemDF).where(uDataDF["Movie_id"]==uItemDF["Movie_id"])
uMovieDF.display()

User_id,Movie_id,Rating,Date,Time,Movie_id,Movie_title
196,242,3,1997-55-04,03:55:49,242,Kolya (1996)
186,302,3,1998-22-04,07:22:22,302,L.A. Confidential (1997)
22,377,1,1997-18-07,07:18:36,377,Heavyweights (1994)
244,51,2,1997-02-27,05:02:03,51,Legends of the Fall (1994)
166,346,1,1998-33-02,05:33:16,346,Jackie Brown (1997)
298,474,4,1998-20-07,02:20:06,474,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
115,265,2,1997-51-03,05:51:28,265,"Hunt for Red October, The (1990)"
253,465,5,1998-34-03,06:34:27,465,"Jungle Book, The (1994)"
305,451,3,1998-20-01,09:20:17,451,Grease (1978)
6,86,3,1997-16-31,09:16:53,86,"Remains of the Day, The (1993)"


### Question 11:  Code Challenge - Perform an Aggregation

Use Apache Spark to do the following:
1. create an aggregate DataFrame, `aggDF` by
  1. extracting the year from the `date` (of the review)
  1. getting the average rating of each film per year as a column named `average_rating`
  1. ordering descending by year and average rating
1. write the resulting dataframe to a table named "movie_by_year_average_rating" in the Default database  
   **note:** use `mode(overwrite)` 

#### Desired Schema
The schema of you resulting DataFrame should be:
- `year INTEGER`
- `movie_title STRING`
- `average_rating DOUBLE`

**NOTE:** Please display the DataFrame `aggDF`.

In [0]:
from pyspark.sql.functions import *

DF1 = uMovieDF.withColumn("Year", split(col("Date"), "-").getItem(0)).select("Year","Movie_title","Rating").groupby("Year","Movie_title").mean()
DF2=DF1.select("Year","Movie_title",col("avg(Rating)").alias("average_rating"))
aggDF=DF2.orderBy(["Year","average_rating"],ascending=False)
aggDF.display(truncate=False)
aggDF.printSchema()
aggDF.write.csv("/FileStore/tables/aggDF/")


spark.sql("Drop TABLE default.movie_by_year_average_rating")
sqlContext.sql("create table default.movie_by_year_average_rating(Year INT, Movie_title STRING, average_rating Float) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n'")
spark.sql("LOAD DATA INPATH '/FileStore/tables/aggDF/' OVERWRITE INTO TABLE default.movie_by_year_average_rating")
#spark.sql("select * from default.movie_by_year_average_rating  where average_rating is Null").show()
#uMovieDF.select("Date","Movie_title","Rating").where(uMovieDF["Movie_title"]=="Spy Hard (1996)").show()
#from pyspark.sql import SQLContext
#sqlContext = new org.apache.spark.sql.SQLContext(sc)


Year,Movie_title,average_rating
1998,Star Kid (1997),5.0
1998,Prefontaine (1997),5.0
1998,"Great Day in Harlem, A (1994)",5.0
1998,Entertaining Angels: The Dorothy Day Story (1996),5.0
1998,The Deadly Cure (1996),5.0
1998,"Visitors, The (Visiteurs, Les) (1993)",5.0
1998,Faust (1994),5.0
1998,Marlene Dietrich: Shadow and Light (1996),5.0
1998,Mina Tannenbaum (1994),5.0
1998,Year of the Horse (1997),4.75


root
 |-- Year: string (nullable = true)
 |-- Movie_title: string (nullable = true)
 |-- average_rating: double (nullable = true)

+----+--------------------+--------------+
|Year|         Movie_title|average_rating|
+----+--------------------+--------------+
|1998|           "Visitors|          null|
|1998|"Great Day in Harlem|          null|
|1998|          "Innocents|          null|
|1998|     "Usual Suspects|          null|
|1998|        "Close Shave|          null|
|1998|   "Whole Wide World|          null|
|1998|"Letter From Deat...|          null|
|1998|              "Maybe|          null|
|1998|     "Wrong Trousers|          null|
|1998|          "Godfather|          null|
|1998|"Shawshank Redemp...|          null|
|1998|      "Cement Garden|          null|
|1998|          "Third Man|          null|
|1998|     "Maltese Falcon|          null|
|1998|      "African Queen|          null|
|1998|"Manchurian Candi...|          null|
|1998|     "Killing Fields|          null|
|1998|   

## Part 4: Fun with JSON

JSON values are typically passed by message brokers such as Kafka or Kinesis in a string encoding. When consumed by a Spark Structured Streaming application, this json must be converted into a nested object in order to be used.

Below is a list of json strings that represents how data might be passed from a message broker.

**Note:** Make sure to run the cell below to retrieve the sample data.

In [0]:
%python


sampleJson = [
 ('{"user":100, "ips" : ["191.168.192.101", "191.168.192.103", "191.168.192.96", "191.168.192.99"]}',), 
 ('{"user":101, "ips" : ["191.168.192.102", "191.168.192.105", "191.168.192.103", "191.168.192.107"]}',), 
 ('{"user":102, "ips" : ["191.168.192.105", "191.168.192.101", "191.168.192.105", "191.168.192.107"]}',), 
 ('{"user":103, "ips" : ["191.168.192.96", "191.168.192.100", "191.168.192.107", "191.168.192.101"]}',), 
 ('{"user":104, "ips" : ["191.168.192.99", "191.168.192.99", "191.168.192.102", "191.168.192.99"]}',), 
 ('{"user":105, "ips" : ["191.168.192.99", "191.168.192.99", "191.168.192.100", "191.168.192.96"]}',), 
]

### Question 12:  Code Challenge - Count the IPs

Use any coding techniques known to you to parse this list of JSON strings to answer the following question:
- how many occurrences of each IP address are in this list?

#### Desired Output
Your results should be this:


| ip | count |
|:-:|:-:|
| `191.168.192.96` | `3` |
| `191.168.192.99` | `6` |
| `191.168.192.100` | `2` |
| `191.168.192.101` | `3` |
| `191.168.192.102` | `2` |
| `191.168.192.103` | `2` |
| `191.168.192.105` | `3` |
| `191.168.192.107` | `3` |

**NOTE:** The order of your results is not important.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Row

list2=[]
for i in sampleJson:
  x=list(i)
  y=x[0].split(': ["')
  for j in y:
    z=j.split('", "')
    for k in z:
      a=k.split('"]')
      for l in a:
        if len(l)==15 or len(l)==14:
          list2.append(l)
rdd1 = sc.parallelize(list2)
row_rdd = rdd1.map(lambda x: Row(x))
df_ip=sqlContext.createDataFrame(row_rdd,['ip']) 
df_ip.select("ip").groupBy("ip").count().orderBy("ip",ascending=True).display()
### below code also works however the column name argument is not possible to assign
#001>>>df_ip=spark.createDataFrame(list2, StringType())
#002>>>print(df_ip)
#003>>>df_ip.select("value").groupBy("value").count().display()

##### pandas created dataframe didnt recognize select ,groupby
#101,102
#101>>>df_ip = pd.DataFrame({'ip':list2}) 
#102>>>df_ip.select("ip").groupBy("ip").count().show()

###### tried creating sql context didnt work as it was considered as object instead of row format
##error:'TypeError: StructType can not accept object '191.168.192.105' in type <class 'str'>'.
#201,202,203
#201>>>schema = StructType([StructField("ip", StringType(), True)])
#202>>>rdd_ip = sc.parallelize(list2)
#203>>>df_ip = sqlContext.createDataFrame(rdd_ip, schema)

ip,count
191.168.192.100,2
191.168.192.101,3
191.168.192.102,2
191.168.192.103,2
191.168.192.105,3
191.168.192.107,3
191.168.192.96,3
191.168.192.99,6


DataFrame[ip: string]


## Part 5: The Databricks API

### Question 13: Conceptual Question - the Databricks API

In 4-5 sentences, please explain what the Databricks API is used for at a high-level.

### Answer:

- The Databricks API is to connect to a single node cluster in community edition
- We have the dbfs as file system and databases
- We can also mount the external filesystem on to Databricks
- we can run Spark SQL, Pyspark, Python on notebooks
- This is industrial wide used platform for big data processing

### Question 14: Conceptual Question - Explain an API Call

In 4-5 sentences, please explain what this API call. Be sure to discuss some key attributes about the cluster.

```
$ curl -n -X POST -H 'Content-Type: application/json'                      \
  -d '{                                                                     \
  "cluster_name": "high-concurrency-cluster",                               \
  "spark_version": "4.2.x-scala2.11",                                       \
  "node_type_id": "i3.xlarge",                                              \
  "spark_conf":{                                                            \
        "spark.databricks.cluster.profile":"serverless",                    \
        "spark.databricks.repl.allowedLanguages":"sql,python,r"             \
     },                                                                     \
     "aws_attributes":{                                                     \
        "zone_id":"us-west-2c",                                             \
        "first_on_demand":1,                                                \
        "availability":"SPOT_WITH_FALLBACK",                                \
        "spot_bid_price_percent":100                                        \
     },                                                                     \
   "custom_tags":{                                                          \
        "ResourceClass":"Serverless"                                        \
     },                                                                     \
       "autoscale":{                                                        \
        "min_workers":1,                                                    \
        "max_workers":2                                                     \
     },                                                                     \
  "autotermination_minutes":10                                              \
}' https://dogfood.staging.cloud.databricks.com/api/2.0/clusters/create '
```

### Answer:

- This is to create cluster with AWS serverless lambda resource
- Associated with programing languages set to execute SQL,Python,R 
- Autoscaling feature of min 1 node max 2 nodes based on 1 node as on demand instance and other as spot instance with 100% bid price
- Also auto termination of idle node with 10 sec heartbeat rate

## Part 6: Security

### Question 15: Conceptual Question - Security on Databricks

Using the Databricks Documentation, what would you recommend to a Databricks and AWS customer for **securely** storing and accessing their data.

### Answer:

- It is good practice to use the AWS S3 to store the data as storage 
- Mount the S3 root folder to dbfs path 
- This makes the efficent model for fast computation along side with storage

# This is the end of the official test. Bonus below!

## Part 7: Bonus: Data Science & Machine Learning

### Question 16: Conceptual Question - A Skewed Feature

One of these lines is the *mean* of this feature. The other is the *median*. Which of these lines is the **mean** - the red line or the black line?

<img width=400px src=https://www.evernote.com/l/AAEycL6CQ0hLi5V5pIo91Ko-Pfk2i0AnGyMB/image.png>

### Answer:

- Black line is the mean

### Question 17: Conceptual Question - Exploratory Data Analysis

The plots below show the distribution of home selling prices differentiated by a few categorical features. Based on these plots, **which of these categorical features** - Property Type, Exterior Quality, or Month Sold - would you expect to be most associated with Price? **Why**?

<img width=1600px src=https://www.evernote.com/l/AAHulkcc20hHSJV6D1udKiwSDCN0S6oV_5YB/image.png>

### Answer:

- Not sure how to read the plots

### Question 18: Conceptual Question - Analyze Model Performance

Consider the following results for a decision tree model against training and testing data sets:

`decision tree regression - train r2 score: 0.9944`  
`decision tree regression - test r2 score:  0.3119`


What is your assessment of this model?

### Answer:

- Not sure what is expected to be answered

### Question 19: Conceptual Question - Model Selection

A series of models has been built using the same training data, but each with a subset of features.

Consider the following results for a series of logistic regression models and then answer this question:

- Which model would you choose and why?
- What other things would you want to look at and why?

| model number | feature subset | logistic regression test accuracy|
|:-:|:-:|:-:|
| 1| feat_1 |	 0.631|
| 2| feat_2 |	 0.552|
| 3| feat_3 |	 0.868|
| 4| feat_4 |	 0.868|
| 5| feat_1, feat_2 |	 0.657|
| 6| feat_1, feat_3 |	 0.947|
| 7| feat_1, feat_4 |	 0.921|
| 8| feat_2, feat_3 |	 0.947|
| 9| feat_2, feat_4 |	 0.973|
| 10| feat_3, feat_4 |	 0.947|
| 11| feat_1, feat_2, feat_3 |	 0.947|
| 12| feat_1, feat_2, feat_4 |	 0.947|
| 13| feat_1, feat_3, feat_4 |	 0.947|
| 14| feat_2, feat_3, feat_4 |	 0.973|
| 15| feat_1, feat_2, feat_3, feat_4 |	 0.973|

### Answer:

- Model 9 is the best, as the regression test accuracy is high and lesser in feature subset
- Model 14 as its second best

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>